# Load data in

In [ ]:
import os
from pathlib import Path

current_dir = Path('./')
data_dir = current_dir / 'speech_commands_v0.01'
train_file_path = data_dir / "train_digit_list.txt"
test_file_path = data_dir / "testing_digit_list.txt"

train_file = open(train_file_path, "r")
training_list = [data_dir / x for x in train_file.read().splitlines()]

test_file = open(test_file_path, "r")
testing_list = [data_dir / x for x in test_file.read().splitlines()]

In [ ]:
import os
from pathlib import Path
import numpy as np

with open('X_train_original.npy', 'rb') as f:
    X_train_org = np.load(f)

with open('X_train_reverb_random.npy', 'rb') as f:
    X_train_reverb = np.load(f)
    
with open('X_test_reverb_random.npy', 'rb') as f:
    X_test_reverb = np.load(f)

In [ ]:
import matplotlib.pyplot as plt
import sidekit.frontend.features as fe
import numpy as np
import scipy.io.wavfile as wav

sample_num = 25
wav.write('./audio/iw_original.wav',  16000, X_train_org[sample_num].astype(np.int16))
wav.write('./audio/iw_reverb.wav', 16000, X_train_reverb[sample_num].astype(np.int16))

# Do 1 data filter

In [ ]:
from pyroomacoustics.denoise.iterative_wiener import apply_iterative_wiener
import soundfile as sf

noisy = sf.read('./audio/iw_reverb.wav')

denoised_signal = apply_iterative_wiener(noisy[0], frame_len=512,
                                         lpc_order=20, iterations=2,
                                         alpha=0.8, thresh=0.01)
sf.write('./audio/iw_fix.wav', denoised_signal, 16000)

In [ ]:
import librosa

def plot_stft(prefix):
    original_file = './audio/{}_original.wav'.format(prefix)
    reverb_file = './audio/{}_reverb.wav'.format(prefix)
    denoise_file = './audio/{}_fix.wav'.format(prefix)
    
    o = librosa.load(original_file, 16000)[0]
    r = librosa.load(reverb_file, 16000)[0]
    d = librosa.load(denoise_file, 16000)[0]
        
    O = librosa.amplitude_to_db(librosa.stft(o), ref=np.max)
    R = librosa.amplitude_to_db(librosa.stft(r), ref=np.max)
    D = librosa.amplitude_to_db(librosa.stft(d), ref=np.max)
    
    return O, R, D

In [ ]:
o, r, d = plot_stft('iw')

In [ ]:
import librosa.display

plt.figure(figsize=(6, 4))
librosa.display.specshow(o, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Original')
plt.savefig('img/iw_original.png')
plt.show()


In [ ]:
import librosa.display

plt.figure(figsize=(6, 4))
librosa.display.specshow(r, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Reverb')
plt.savefig('img/iw_reverb.png')
plt.show()

In [ ]:
import librosa.display

plt.figure(figsize=(6, 4))
librosa.display.specshow(d, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Iterative Weiner Filter')
plt.savefig('img/iw_fix.png')
plt.show()

# Do all data filter

In [ ]:
from tqdm.notebook import tnrange
from pyroomacoustics.denoise.iterative_wiener import apply_iterative_wiener

train_data = []
for i in tnrange(len(X_train_reverb)):
    denoised_signal = apply_iterative_wiener(X_train_reverb[i].flatten(), frame_len=512, lpc_order=20, iterations=2, alpha=0.8, thresh=0.01)
    train_data.append(denoised_signal)
    
train_data = np.asarray(train_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "Iwt_train_reverb.npy", train_data)

In [ ]:
from tqdm.notebook import tnrange
from pyroomacoustics.denoise.iterative_wiener import apply_iterative_wiener

test_data = []
for i in tnrange(len(X_test_reverb)):
    denoised_signal = apply_iterative_wiener(X_test_reverb[i].flatten(), frame_len=512, lpc_order=20, iterations=2, alpha=0.8, thresh=0.01)
    test_data.append(denoised_signal)
    
test_data = np.asarray(test_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "Iwt_test_reverb.npy", test_data)